<a href="https://colab.research.google.com/github/SRARNAB7/End-to-End-Medical-Chat-Bot/blob/main/Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y unrar


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
import os

zip_path = "/content/patient_medications.rar"
extract_path = "/content/extracted_data"

# Make sure the folder exists
os.makedirs(extract_path, exist_ok=True)

# Extract using unrar
!unrar x "{zip_path}" "{extract_path}/"



UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/patient_medications.rar

Extracting  /content/extracted_data/patient_medications.csv                1%  2%  4%  5%  7%  8%  9% 11% 12% 14% 15% 17% 18% 19% 21% 22% 24% 25% 26% 28% 29% 31% 32% 34% 35% 36% 38% 39% 41% 42% 44% 45% 46% 48% 49% 51% 52% 53% 55% 56% 58% 59% 61% 62% 63% 65% 66% 68% 69% 70% 72% 73% 75% 76% 78% 79% 80% 82% 83% 85% 86% 88% 89% 90% 92% 93% 95% 96% 97% 99%100%  OK 
All OK


In [3]:
import pandas as pd

file_path = "/content/extracted_data/patient_medications.csv"

chunksize = 100000
chunks = []

for chunk in pd.read_csv(file_path, chunksize=chunksize, low_memory=False):
    chunks.append(chunk)

df = pd.concat(chunks, ignore_index=True)

print(df.shape)
print(df.head())


(27961524, 18)
   ENC_TYPE_C         ENC_TYPE_NM            LOG_ID               MRN  \
0           3  Hospital Encounter  f51e86014fd956d3  bd51e3b61ead68a6   
1           3  Hospital Encounter  f51e86014fd956d3  bd51e3b61ead68a6   
2           3  Hospital Encounter  f51e86014fd956d3  bd51e3b61ead68a6   
3           3  Hospital Encounter  f51e86014fd956d3  bd51e3b61ead68a6   
4           3  Hospital Encounter  f51e86014fd956d3  bd51e3b61ead68a6   

         ORDERING_DATE ORDER_CLASS_NM  MEDICATION_ID  \
0  2020-08-04 00:00:00      Inpatient           4424   
1  2020-08-04 00:00:00      Inpatient           4424   
2  2020-08-04 00:00:00      Inpatient           4424   
3  2020-08-04 00:00:00      Inpatient          19177   
4  2020-08-04 00:00:00      Inpatient          19177   

                               DISPLAY_NAME  \
0  levothyroxine (SYNTHROID) tablet 125 mcg   
1  levothyroxine (SYNTHROID) tablet 125 mcg   
2  levothyroxine (SYNTHROID) tablet 125 mcg   
3       atorvastatin 

In [ ]:
# Drop specific columns
df_pt_medications = df.drop(
    columns=["DISPLAY_NAME", "MEDICATION_NM"],
    errors="ignore"   # avoid error if a column is missing
)

In [1]:
import pandas as pd

# 1. Normalize only string (object) columns: strip spaces + uppercase
for col in df_pt_medications.select_dtypes(include=["object"]).columns:
    df_pt_medications[col] = df_pt_medications[col].str.strip().str.upper()

# 2. Ensure datetime columns are real datetimes (so dedup works properly)
for dcol in ["START_DATE","END_DATE","ORDERING_DATE","MED_ACTION_TIME"]:
    if dcol in df_pt_medications.columns:
        df_pt_medications[dcol] = pd.to_datetime(df_pt_medications[dcol], errors="coerce")

# 3. Drop duplicates based on full subset (includes MEDICATION_ID as int)
subset_cols = [
    "ENC_TYPE_C","ENC_TYPE_NM","LOG_ID","MRN","ORDERING_DATE","ORDER_CLASS_NM",
    "MEDICATION_ID","START_DATE","END_DATE",
    "ORDER_STATUS_NM","RECORD_TYPE","MAR_ACTION_NM","MED_ACTION_TIME",
    "ADMIN_SIG","DOSE_UNIT_NM","MED_ROUTE_NM"
]
df_pt_medications = (
    df_pt_medications
    .drop_duplicates(subset=subset_cols, keep="first")
    .reset_index(drop=True)
)


# 5. Verify
print(df_pt_medications.head())
print("Final shape:", df_pt_medications.shape)


NameError: name 'df_pt_medications' is not defined